In [1]:
import pandas as pd
import numpy as np

Импортируем нужные библиотеки

In [2]:
def get_sim(item_id):
    rat = [(other_item_id, np.linalg.norm(H[:, item_id - 1] - H[:, other_item_id - 1]))
           for other_item_id in ratings["movie_id"].unique()]
    items_ratings = sorted(rat, key=lambda x: x[1])
    return items_ratings[:10]


def get_recom(user_id):
    unique_movies = set(movies.to_numpy())
    for k in range(len(ratings_np)):
        if ratings_np[k][0] == user_id:
            unique_movies.remove(ratings_np[k][1])
    unique_movies_arr = []
    for t in unique_movies:
        unique_movies_arr.append(t)
    new_items_ratings = (W @ H)[user_id - 1][np.array(unique_movies_arr) - 1]
    items_ratings = sorted(list(zip(unique_movies, new_items_ratings)), key=lambda x: x[1], reverse=True)
    return items_ratings[:10]

Два дополнительных метода, чтобы считать симилары для конкретного айтема и рекомендации для конкретного юзера

In [3]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

movie_info = pd.read_csv('movies.dat', delimiter='::', header=None,
                         names=['movie_id', 'name', 'category'], engine='python')

users = ratings["user_id"]
movies = ratings["movie_id"]
ratings_np = ratings.to_numpy()
users_size = users.max()
items_size = movies.max()

Подгружаем наш датасет

In [4]:
K = 30
eps = 0.01
lam = 1e-5
count_iter = 10000000

Подбираем гиперпараметры модели

In [5]:
W = np.random.uniform(0, 1 / np.sqrt(K), size=(users_size, K))
H = np.random.uniform(0, 1 / np.sqrt(K), size=(K, items_size))

Инициализируем матрицы юзеров (W) и айтемов (H)

In [6]:
for t in range(count_iter):
    k = np.random.randint(len(ratings_np))
    i = ratings_np[k][0] - 1
    j = ratings_np[k][1] - 1
    l = ratings_np[k][2]
    error = W[i, :] @ H[:, j] - l
    W[i, :] = W[i, :] - eps * (error * H[:, j].T + lam * W[i, :])
    H[:, j] = H[:, j] - eps * (error * W[i, :].T + lam * H[:, j])

Обучаем нашу модель на explicit данных

In [7]:
get_similars = lambda item_id, model: [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() for x in model]
for i in get_similars(1, get_sim(1)):
    print(i)

0    Toy Story (1995)
3045    Toy Story 2 (1999)
584    Aladdin (1992)
3090    Fantasia 2000 (1999)
2618    Tarzan (1999)
922    Father of the Bride (1950)
2496    King and I, The (1956)
1636    Truman Show, The (1998)
2011    Lady and the Tramp (1955)
3400    Inherit the Wind (1960)


Симилары действительно симилары

In [8]:
get_recommendations = lambda user_id, model: [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() for x in model]
for i in get_recommendations(4, get_recom(4)):
    print(i)

523    Schindler's List (1993)
847    Godfather, The (1972)
740    Dr. Strangelove or: How I Learned to Stop Worr...
1203    Godfather: Part II, The (1974)
1189    To Kill a Mockingbird (1962)
2327    Shakespeare in Love (1998)
3360    Creature Comforts (1990)
900    Casablanca (1942)
1186    Lawrence of Arabia (1962)
1176    One Flew Over the Cuckoo's Nest (1975)


Рекомендации тоже вышли неплохие